<a target="_blank" href="https://colab.research.google.com/github/Jasonchen0317/CSGY-6613-Assignment/blob/main/assignment-3/assignment-3.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Part 1: Download videos and get frames

For downloading videos, first put the link of the videos in video_list. I used pytube library for downloading the videos, which will be saved in the 'video' folder. Next, I used cv2 to get the frames of the videos, and save the frames in 'dataset/test', 'dataset' directory includes training, validation, and testing dataset.

In [1]:
!pip install pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.9 MB/s eta 0:00:00


In [1]:
from pytube import YouTube
import cv2
import os

import numpy as np
import tqdm
import pandas as pd


In [2]:
def download_video(video_list):
    if not os.path.exists('video'):
        os.makedirs('video')
        
    for i in range(len(video_list)):
        #get link
        link = video_list[i]
        yt = YouTube(link)
        yt.bypass_age_gate()
        #Download video
        filename = "%s.mp4" % str(i)
        yt.streams.filter(res="360p").first().download(filename=filename, output_path='video')

In [3]:
video_list = [
    'https://www.youtube.com/watch?v=DhmZ6W1UAv4',
    'https://www.youtube.com/watch?v=YrydHPwRelI'
]

In [4]:
download_video(video_list)

In [5]:
v_list = os.listdir('video')
for i in range(len(v_list)):
  vid = i
  capture = cv2.VideoCapture(f'video/{vid}.mp4')
  length = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
  print( length )

4941
15409


In [6]:
if not os.path.exists('dataset/test'):
   os.makedirs('dataset/test')

In [7]:
DIR_TEST = 'dataset/test'
for i in range(len(v_list)):
  vid = i
  capture = cv2.VideoCapture(f'video/{vid}.mp4')
  frameNr = 0
  success, frame = capture.read()
  while (success):
      cv2.imwrite(f'{DIR_TEST}/video_{vid}_frame_{frameNr}.jpg', frame)
      success, frame = capture.read()
      frameNr = frameNr+1

  capture.release()


# Drone Object Detection

Training and Validation Dataset: https://www.kaggle.com/datasets/dasmehdixtr/drone-dataset-uav , I picked about 500 images for training, and about 100 images for validating.

The other python files are for drone detection: 

    config.py includes the configurations such as dataset paths, hyperparameters for the model, classes, etc. 

    custom_utils.py includes functions for saving trained models, saving plots, transforming images, etc. 

    datasets.py is used is for converting raw datas to datasets that fits the input of the model. 

    model.py builds the model for this task, which is the fasterrcnn_resnet50_fpn from torch. 

    train.py will be run to start training the model. The output includes best model and loss plots. 

    detect.py will be run to detect the drones from the frames collected in part 1. The outputs is saved in the 'detections' directory.

Some Results of the detected frames are shown in the readme file.

In [1]:
import torch
torch.cuda.is_available()

True

In [5]:
torch.__version__
torch.cuda.get_device_properties(0)

_CudaDeviceProperties(name='NVIDIA GeForce RTX 4050 Laptop GPU', major=8, minor=9, total_memory=6140MB, multi_processor_count=20)

In [3]:
!nvidia-smi

Thu Nov  9 12:25:00 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 536.67                 Driver Version: 536.67       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4050 ...  WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   52C    P0              22W /  44W |   5922MiB /  6141MiB |     98%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
if not os.path.exists('outputs'):
   os.makedirs('outputs')

In [1]:
!python train.py

Number of training samples: 480

c:\Users\jason\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\jason\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
libpng warning: iCCP: known incorrect sRGB profile

Loss: 0.1808: 100%|██████████| 60/60 [09:05<00:00,  8.10s/it]libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile

Loss: 0.1808: 100%|██████████| 60/60 [09:06<00:00,  9.11s/it]

Loss: 0.


Number of validation samples: 123


EPOCH 1 of 10
Training
Validating
Epoch #1 train loss: 0.257
Epoch #1 validation loss: 0.175
Took 10.338 minutes for epoch 0

Best validation loss: 0.17512262146919966

Saving best model for epoch: 1

SAVING PLOTS COMPLETE...

EPOCH 2 of 10
Training
Validating
Epoch #2 train loss: 0.171
Epoch #2 validation loss: 0.141
Took 9.798 minutes for epoch 1

Best validation loss: 0.14142183726653457

Saving best model for epoch: 2

SAVING PLOTS COMPLETE...

EPOCH 3 of 10
Training
Validating
Epoch #3 train loss: 0.143
Epoch #3 validation loss: 0.122
Took 9.989 minutes for epoch 2

Best validation loss: 0.12188163725659251

Saving best model for epoch: 3

SAVING PLOTS COMPLETE...

EPOCH 4 of 10
Training
Validating
Epoch #4 train loss: 0.123
Epoch #4 validation loss: 0.118
Took 10.341 minutes for epoch 3

Best validation loss: 0.11758043617010117

Saving best model for epoch: 4

SAVING PLOTS COMPLETE...

EPOCH 5 of 10
Training
Validating
Epoch #5 train loss: 0.

In [ ]:
if not os.path.exists('detections'):
   os.makedirs('detections')

In [ ]:
!python detect.py